In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'/home/lucky/Documents/projects/MyProject/smoke_detection'

In [4]:
import tensorflow as tf

2023-12-30 23:51:15.106814: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 23:51:15.141576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 23:51:15.141608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 23:51:15.142700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 23:51:15.148229: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 23:51:15.148940: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    test_path: Path
    
    img_size: list
    epochs: int
    batch_size: int
    learning_rate: float

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        config = self.config.evaluation
        eval_config = EvaluationConfig(
            model_path=config.model_path,
            test_path=config.test_path,
            img_size=self.params.IMAGE_SIZE,
            epochs=self.params.epochs,
            batch_size = self.params.BATCH_SIZE,
            learning_rate=self.params.LEARNING_RATE
        )
        return eval_config

In [14]:
ConfigurationManager().get_validation_config()

[2023-12-30 23:59:26,599: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-30 23:59:26,602: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 23:59:26,604: INFO: common: created directory at: artifacts]


EvaluationConfig(model_path='artifacts/training/trained_model.keras', test_path='artifacts/data_ingestion/smoke_data/Testing/Testing', img_size=BoxList([224, 224]), epochs=5, batch_size=32, learning_rate=0.001)

In [17]:
import os
import tensorflow as tf
import keras
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

In [24]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.test_gen = None

    @staticmethod
    def extract_from_path(path)-> []:
        full_path = []
        for i in sorted(os.listdir(path)):
            full_path.append(os.path.join(path, i))
        return full_path

    @staticmethod
    def createDataFrame(image_paths):
        labels = [path.split('/')[-1].split('_')[0] for path in image_paths]
        df = pd.DataFrame({'path': image_paths, 'label': labels})
        df['label_id'] = df['label'].apply(lambda x: 1 if x == 'smoking' else 0)
        return df
    
    def imgPreProcessing(self, image, label):
        img = tf.io.read_file(image)
        img = tf.io.decode_jpeg(img, channels = 3)
        img = tf.image.resize(img, size = (self.config.img_size))
        return img, label
    
    def loadDataset(self, df:pd.DataFrame):
        dataset = tf.data.Dataset.from_tensor_slices((df['path'], df['label_id']))
        return (dataset
                     .map(self.imgPreProcessing)
                    .shuffle(self.config.batch_size * 20)
                     .batch(self.config.batch_size))
        
    def _test_gen(self):
        test_images = self.extract_from_path(self.config.test_path)
        test_images_df = self.createDataFrame(test_images)
        self.test_gen = self.loadDataset(test_images_df)
        return self.test_gen
        

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        model = self.load_model(self.config.model_path)
        self._test_gen()
        self.score = model.evaluate(self.test_gen)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [25]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-12-31 00:06:41,543: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-31 00:06:41,546: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-31 00:06:41,547: INFO: common: created directory at: artifacts]
7/7 [==============================] - 6s 493ms/step - loss: 0.1236 - accuracy: 0.9509
[2023-12-31 00:06:54,506: INFO: common: json file saved at: scores.json]
